This script merges all term-specific predictors constructed in previous steps together, for all non-first-term observations.

In [1]:
import pandas as pd
import numpy as np

In [2]:
all_vccsid_strm = pd.read_stata("~\\Box Sync\\Clickstream\\data\\all_vccsid_strm.dta")

In [3]:
d1 = pd.read_stata('~\\Box Sync\\Clickstream\\data\\age_gpa_earn.dta')
d2 = pd.read_stata('~\\Box Sync\\Clickstream\\data\\four_additional_predictors.dta')
d3 = pd.read_stata('~\\Box Sync\\Clickstream\\data\\twelve_additional_predictors.dta')
d4 = pd.read_stata('~\\Box Sync\\Clickstream\\data\\trend_predictors.dta')
d5 = pd.read_stata('~\\Box Sync\\Clickstream\\data\\ever_dually_enrolled.dta')

In [4]:
merged = all_vccsid_strm.merge(d1, how='inner', on=['vccsid','strm']).\
merge(d2, how='inner', on=['vccsid','strm']).\
merge(d3, how='inner', on=['vccsid','strm']).\
merge(d4, how='inner', on=['vccsid','strm']).\
merge(d5, how='inner', on=['vccsid','strm'])

In [5]:
merged.shape

(10801, 21)

In [11]:
impute_list_1 = set(["cum_gpa", "overall_prop_comp", "prop_comp_sd"])

In [12]:
def impute(train, test):
    for p in impute_list_1:
        avg_p = np.nanmean(train[p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    return train, test               

In [13]:
train_df, test_df = impute(merged[merged.strm < 2212], merged[merged.strm == 2212])

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
merged_new = pd.concat([train_df, test_df])
merged_new.to_csv("~\\Box Sync\\Clickstream\\data\\term_specific_predictors.csv", index=False)

In [19]:
assert pd.isnull(merged_new).any().any() == False
merged_new.shape

(10801, 21)